## IPython + jit decorator

We can define jit function in IPython (and Jupyter, which uses IPython). The only limitation is that the type variables have to be defined in the same cell as the function.

In [1]:
import numpy as np

from transonic import jit

In [2]:
# transonic import numpy as np

@jit
def laplace_pythran(image):
    """Laplace operator in NumPy for 2D images."""
    laplacian = (
        image[:-2, 1:-1] + image[2:, 1:-1] + image[1:-1, :-2] + image[1:-1, 2:]
        - 4*image[1:-1, 1:-1]
    )
    thresh = np.abs(laplacian) > 0.05
    return thresh

In [3]:
from skimage.data import astronaut
from skimage.color import rgb2gray

image = astronaut()
image = rgb2gray(image)

laplace_pythran(image)

INFO     write Pythran signature in file /home/users/augier3pi/.transonic/__jit__/__ipython__2823fa65cda0fe3f58b60391a9d2e13b/laplace_pythran.pythran with types
['float64[:, :]']


array([[False,  True,  True, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True,  True, False]])

After a warmup period, the function `laplace_pythran` is going to be very efficient.